# 准备数据

In [2]:
# 下载数据并解压
!wget -nc "https://labfile.oss.aliyuncs.com/courses/1283/renthop_train.json.gz"
!gunzip "renthop_train.json.gz"

'wget' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���
'gunzip' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���


In [3]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [4]:
df = pd.read_json('renthop_train.json')
df.head()

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address,interest_level
4,1.0,1,8579a0b0d54db803821a35a4a615e97a,2016-06-16 05:55:27,Spacious 1 Bedroom 1 Bathroom in Williamsburg!...,145 Borinquen Place,"[Dining Room, Pre-War, Laundry in Building, Di...",40.7108,7170325,-73.9539,a10db4590843d78c784171a107bdacb4,[https://photos.renthop.com/2/7170325_3bb5ac84...,2400,145 Borinquen Place,medium
6,1.0,2,b8e75fc949a6cd8225b455648a951712,2016-06-01 05:44:33,BRAND NEW GUT RENOVATED TRUE 2 BEDROOMFind you...,East 44th,"[Doorman, Elevator, Laundry in Building, Dishw...",40.7513,7092344,-73.9722,955db33477af4f40004820b4aed804a0,[https://photos.renthop.com/2/7092344_7663c19a...,3800,230 East 44th,low
9,1.0,2,cd759a988b8f23924b5a2058d5ab2b49,2016-06-14 15:19:59,**FLEX 2 BEDROOM WITH FULL PRESSURIZED WALL**L...,East 56th Street,"[Doorman, Elevator, Laundry in Building, Laund...",40.7575,7158677,-73.9625,c8b10a317b766204f08e613cef4ce7a0,[https://photos.renthop.com/2/7158677_c897a134...,3495,405 East 56th Street,medium
10,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,[],40.7145,7211212,-73.9425,5ba989232d0489da1b5f2c45f6688adc,[https://photos.renthop.com/2/7211212_1ed4542e...,3000,792 Metropolitan Avenue,medium
15,1.0,0,bfb9405149bfff42a92980b594c28234,2016-06-28 03:50:23,Over-sized Studio w abundant closets. Availabl...,East 34th Street,"[Doorman, Elevator, Fitness Center, Laundry in...",40.7439,7225292,-73.9743,2c3b41f588fbb5234d8a1e885a436cfa,[https://photos.renthop.com/2/7225292_901f1984...,2795,340 East 34th Street,low


在实践中，很少有数据是以矩阵形式保存、可以直接使用的，这就是为什么需要对数据进行特征提取。首先看看在一些常见的数据类型中特征提取是如何进行的。

**文本数据**


- 在处理文本之前，必须对文本进行切分词（tokenzie）操作，也就是将文本切分为单元（token）。在最简单的情形下，一个 token 就是一个单词。但直接按单词切分可能会损失一些信息，比如「Santa Barbara」应该是一个整体，却被切分为 2 个 token。现成的分词器（tokenizer）会考虑语言的特性，但也会出错，特别是当你处理特定来源的文本时（报纸、俚语、误拼、笔误）。
- 接下来需要正则化数据。对文本而言，这涉及词干提取（stemming）和词形还原（lemmatization）方法。这两个方法是词形规范化的两类重要方式，都能够达到有效归并词形的目的，二者既有联系也有区别。两者的区别可以参考《Introduction to Information Retrieval》一书的  Stemming and lemmatization 一节。
- 当文档被转换为单词序列之后，就可以用向量表示它。最简单的方法是词袋（Bag of Words）模型：创建一个长度等于字典的向量，计算每个单词出现在文本中的次数，然后将次数放入向量对应的位置中。

下面用代码来表述词袋模型。



In [5]:

texts = ['i have a cat',
         'you have a dog',
         'you and i have a cat and a dog']
vocabulary  = list(enumerate(set([word for setence in texts for word in setence.split()])))
print('Vocabulary:', vocabulary)


def vectorize(text):
    vector = np.zeros(len(vocabulary))
    for i,word in vocabulary:
        num = 0
        for w in text:
            if w==word:
                num+=1
            if num:
                vector[i] = num
    return vector
print('Vectors:')
for sentence in texts:
    print(vectorize(sentence.split()))

Vocabulary: [(0, 'cat'), (1, 'have'), (2, 'a'), (3, 'you'), (4, 'i'), (5, 'dog'), (6, 'and')]
Vectors:
[1. 1. 1. 0. 1. 0. 0.]
[0. 1. 1. 1. 0. 1. 0.]
[1. 1. 2. 1. 1. 1. 2.]


当使用词袋模型时，文本中的单词顺序信息会丢失，这意味着向量化之后，“i have no cows”（我没有牛）和“no, i have cows”（没，我有牛）会变得一样，尽管事实上它们的意思截然相反。

为了避免这个问题，可以转而使用 N-Gram 模型。下面载入相关库，建立 N-Gram 模型。

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(ngram_range=(1,1))
vect.fit_transform(['no i have cows', 'i have no cows']).toarray()
vect.vocabulary_

{'no': 2, 'have': 1, 'cows': 0}

In [7]:
# 改变参数
vect = CountVectorizer(ngram_range=(1, 2)) # 一个词到两个词
vect.fit_transform(['no i have cows', 'i have no cows']).toarray()
vect.vocabulary_

{'no': 4,
 'have': 1,
 'cows': 0,
 'no have': 6,
 'have cows': 2,
 'have no': 3,
 'no cows': 5}

In [8]:
from scipy.spatial.distance import euclidean
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(ngram_range=(3, 3), analyzer='char_wb')

n1, n2, n3, n4 = vect.fit_transform(
    ['andersen', 'petersen', 'petrov', 'smith']).toarray()

euclidean(n1, n2), euclidean(n2, n3), euclidean(n3, n4)

(np.float64(2.8284271247461903),
 np.float64(3.1622776601683795),
 np.float64(3.3166247903554))

**图像数据**

**特征转换**

对有些算法而言，特征转换是关键，而对另一些算法而言，该转换则毫无帮助。不是所有人都有能力或者愿意去进行特征转换，这是为什么决策树及其变体（随机森林、梯度提升）变得更加流行，因为它们在异常分布上的鲁棒性很好

最简单的转换是标准缩放（Standard Scaling），又称 Z 值标准化（Z-score normalization）：

注意，严格意义上说，标准缩放并不生成正态分布，下面验证这一结论，首先载入相关库。



In [9]:
from sklearn.preprocessing import StandardScaler
from scipy.stats import beta
from scipy.stats import shapiro
import numpy as np

使用夏皮罗 - 威尔克检验（Shapiro–Wilk test）查看数据是否符合正态分布，首先输出原数据的检测统计量和假设检验所得的 p 值。

In [10]:
data = beta(1,10).rvs(1000).reshape(-1,1) # beta(a,b)，期望值是a/(a+b)
shapiro(data)

ShapiroResult(statistic=np.float64(0.8695891601383847), pvalue=np.float64(4.605426500642631e-28))

rvs(1000) 是从概率分布中抽取 1000 个随机样本的标准方法，广泛应用于数据模拟、统计分析和机器学习领域‌。其具体分布形态由所属概率分布参数（如 Beta 分布的 $α,β$)
决定‌

In [11]:
shapiro(StandardScaler().fit_transform(data)) # 这个是标准化的函数

ShapiroResult(statistic=np.float64(0.8695891601383854), pvalue=np.float64(4.605426500643258e-28))

夏皮罗 - 威尔克检验计算所得的 p 值说明原数据和经标准缩放处理后的数据都不符合正态分布。

不过，某种程度上而言，标准缩放能为离散值提供一些帮助。下面使用 StandarScaler() 方法对离散值进行标准缩放

In [12]:
data = np.array([1, 1, 0, -1, 2, 1, 2, 3, -2, 4, 100]
                ).reshape(-1, 1).astype(np.float64)
StandardScaler().fit_transform(data)

array([[-0.31922662],
       [-0.31922662],
       [-0.35434155],
       [-0.38945648],
       [-0.28411169],
       [-0.31922662],
       [-0.28411169],
       [-0.24899676],
       [-0.42457141],
       [-0.21388184],
       [ 3.15715128]])

In [13]:
# 用公式
(data - data.mean()) / data.std()

array([[-0.31922662],
       [-0.31922662],
       [-0.35434155],
       [-0.38945648],
       [-0.28411169],
       [-0.31922662],
       [-0.28411169],
       [-0.24899676],
       [-0.42457141],
       [-0.21388184],
       [ 3.15715128]])

另一个相当流行的选项是极小极大缩放（MinMax Scaling），将所有数据点纳入一个预先规定的区间（通常是 [0,1]）。


$$
X \ast \text{norm} = \frac{X - X \ast \min}{X \ast \max - X \ast \min}
$$


In [14]:
from sklearn.preprocessing import MinMaxScaler

MinMaxScaler().fit_transform(data)

array([[0.02941176],
       [0.02941176],
       [0.01960784],
       [0.00980392],
       [0.03921569],
       [0.02941176],
       [0.03921569],
       [0.04901961],
       [0.        ],
       [0.05882353],
       [1.        ]])

如果某些数据并非正态分布，但可以由  对数正态分布 刻画，那么这些数据很容易就能转换为正态分布，下面生成可以用对数正态分布来刻画的数据，并查看其是否符合正态分布。

In [24]:
from scipy.stats import lognorm
data = lognorm(s=1).rvs(1000)
shapiro(data)

ShapiroResult(statistic=np.float64(0.6289289874473567), pvalue=np.float64(6.827855400460857e-42))

In [20]:
shapiro(np.log(data))

ShapiroResult(statistic=np.float64(0.9989223873068106), pvalue=np.float64(0.8305589601264032))

原假设与检验逻辑‌
Shapiro-Wilk检验的原假设是‌数据服从正态分布‌‌。当p值大于显著性水平（通常取0.05）时，无法拒绝原假设，即认为数据符合正态分布。本例中p值（0.7582）显著高于0.05，因此接受原假设‌。

统计量的意义‌
Shapiro-Wilk统计量的取值范围为0到1，‌越接近1表示数据分布与正态分布的拟合程度越高‌‌。本例中统计量约为0.9988，接近1，进一步支持数据服从正态分布的结论‌。

对数正态分布适用于描述薪水、安保费用、城区人口、网络文章评论等信息。然而，底层分布不一定非得是对数正态分布才能应用这一过程，它可以被应用于任何右重尾的分布。此外，还可以尝试使用其他类似的转换，这类转换的例子包括 Box-Cox 转换（对数转换是 Box-Cox 转换的一个特例）和 Yeo-Johnson 转换（将应用范围扩展至负数）。此外，也可以尝试在特征上加上一个常量：np.log (x + const)。

In [ ]:
import statsmodels.api as sm